This notebook is intended for those who want a gentle introduction to the Evaluation Metric for this competition. 

I created this notebook to decomose the Evaluation Metric, M, and its components, D and G.

In [1]:
import pandas as pd

Let's create a simple benchmark to calculate the Evaluation Metric. We are going to use *P_2* column in train_data.csv to calculate *y_pred* and train_labels.csv to create *y_true*.

In [2]:
train_data = pd.read_csv('../input/amex-default-prediction/train_data.csv', index_col='customer_ID', usecols=['customer_ID', 'P_2'])
train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv', index_col='customer_ID')

In [3]:
train_data.head()

,P_2
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.938469
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.936665
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.954180
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.960384
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.947248


In [4]:
train_labels.head()

,target
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0


Since there are a couple of rows for each *customer_id*, we are going to group them and then calculate the mean of *P_2* column as our prediction.

In [5]:
ave_p2 = (train_data.groupby('customer_ID').mean().rename(columns={'P_2': 'prediction'}))

# Scale the mean P_2 by the max value and take the compliment
ave_p2['prediction'] = 1.0 - (ave_p2['prediction'] / ave_p2['prediction'].max())

ave_p2.head()

,prediction
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.075415
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.109083
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.130237
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.406957
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.117143


In [6]:
y_true = train_labels.copy()
y_pred = ave_p2.copy()

The evaluation metric, **M**, for this competition is the mean of two measures of rank ordering: Normalized Gini Coefficient, **G**, and default rate captured at 4%, **D**.

*M = 0.5 x (G + D)*

In the following, we will calculate each component in detail.

##### Calculating Parameter D (default rate captured at 4%):
The default rate captured at 4% is the percentage of the positive labels (defaults) captured within the highest-ranked 4% of the predictions, and represents a Sensitivity/Recall statistic.

Let's break down the steps to caluclate D:

In [7]:
# Create a df dataframe by concatinating y_true, y_pred and sorting the rows based on y_pred in a descending order.
df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
df

,target,prediction
customer_ID,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066
...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN


In [8]:
# Create a 'weight' column in df with values of 1 for y_true = 0, and 20 for y_true = 0.
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
df

,target,prediction,weight
customer_ID,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1
...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20


In [9]:
# Calculate four_pct_cutoff variable as 4 percent of the sum of all values in 'weight' column.
four_pct_cutoff = int(0.04 * df['weight'].sum())
four_pct_cutoff

276821

In [10]:
# Create 'weight_cumsum' column in df to calculate cumulative sum of weights in 'weight' column.
df['weight_cumsum'] = df['weight'].cumsum()
df

,target,prediction,weight,weight_cumsum
customer_ID,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,3
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,4
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,5
...,...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20,6920448
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20,6920468
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20,6920488


In [11]:
# Create df_cutoff dataframe based on filtering the portion of descending sorted y_pred which has lower weight_cumsum of four_pct_cutoff.
df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
df_cutoff

,target,prediction,weight,weight_cumsum
customer_ID,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,3
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,4
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,5
...,...,...,...,...
e1218aab708625c38cdcca2475f52d9e77d28629d39ef0e5af2095d04841686d,0,0.661767,20,276802
18be41f7bfcaf589a9640dea4f87a4bec4f8a4a576f64c2e5221190d0d737b8d,1,0.661765,1,276803
0f6450f2ff8b11be054b20b04c9f33266b3eac647a3bfd8dbe47643deabab31f,1,0.661761,1,276804


In [12]:
# Calculate the ratio of the number of y_true = 1 in df_cutoff to number of y_true = 1 in the input dataframe.
d = (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
d

0.35181943649644865

##### Calculating Weighted Gini:

Weighted Gini is required to calculate normalized weighted Gini (G parameter).

Let's break down the steps to caluclate Weighted Gini:

In [13]:
# Create df dataframe by concatinating y_true, y_pred and sorting the rows based on y_pred in a descending order.
df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
df

,target,prediction
customer_ID,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066
...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN


In [14]:
# Create 'weight' column in df with values of 1 for y_true = 0, and 20 for y_true = 0.
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
df

,target,prediction,weight
customer_ID,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1
...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20


In [15]:
# Create 'random' column in df that has cumulative sum of 'weight' column divided by sum of 'weight' column.
df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
df

,target,prediction,weight,random
customer_ID,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1.444976e-07
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2.889953e-07
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,4.334929e-07
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,5.779906e-07
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,7.224882e-07
...,...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20,9.999884e-01
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20,9.999913e-01
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20,9.999942e-01


In [16]:
# Define total_pos variable by the sum of all y_true values to their corresponding 'weight' values.
total_pos = (df['target'] * df['weight']).sum()
total_pos

118828

In [17]:
# Create 'cum_pos_found' column in df by calculating cumulative sum of the multiplication of y_true and their corresponding 'weight' values.
df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
df

,target,prediction,weight,random,cum_pos_found
customer_ID,,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1.444976e-07,1
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2.889953e-07,2
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,4.334929e-07,3
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,5.779906e-07,4
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,7.224882e-07,5
...,...,...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20,9.999884e-01,118828
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20,9.999913e-01,118828
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20,9.999942e-01,118828


In [18]:
# Create 'lorentz' column by dividing 'cum_pos_found' values by total_pos value.
df['lorentz'] = df['cum_pos_found'] / total_pos
df

,target,prediction,weight,random,cum_pos_found,lorentz
customer_ID,,,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1.444976e-07,1,0.000008
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2.889953e-07,2,0.000017
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,4.334929e-07,3,0.000025
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,5.779906e-07,4,0.000034
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,7.224882e-07,5,0.000042
...,...,...,...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20,9.999884e-01,118828,1.000000
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20,9.999913e-01,118828,1.000000
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20,9.999942e-01,118828,1.000000


In [19]:
# Create 'gini' column by ('lorentz' - 'random') * 'weight' values
df['gini'] = (df['lorentz'] - df['random']) * df['weight']
df

,target,prediction,weight,random,cum_pos_found,lorentz,gini
customer_ID,,,,,,,
573d168b98078aa79c4f9db6b8751e23b746905e698c379dafbbe5bfd5629ce0,1,1.334929,1,1.444976e-07,1,0.000008,8.271027e-06
3812caed075be5a3602557212e1124fb9f7ccb08a0a1455ad73aeddbb460719c,1,1.326789,1,2.889953e-07,2,0.000017,1.654205e-05
1bacd12b550758715eb5788f10937ec51786680f51398686d10966e7c774febc,1,1.318257,1,4.334929e-07,3,0.000025,2.481308e-05
6ca0778956b26b6fd5304a7f70d75f571d0af2285069f62a2b5d68c2b57dccb9,1,1.300520,1,5.779906e-07,4,0.000034,3.308411e-05
66fbfe312c6d88f4f5dcc3cd2983d06c7ea3fe6c47c232f4e3794b1ad14f312b,1,1.288066,1,7.224882e-07,5,0.000042,4.135514e-05
...,...,...,...,...,...,...,...
ff9010b669f69e5e78408676de5a5c7a896bccc4d4918d31f5f15ae921b83b80,0,NaN,20,9.999884e-01,118828,1.000000,2.311963e-04
ffab7e49cf10a680969be4cb531c3dbe200113737466dd2933e5a03c37b82252,0,NaN,20,9.999913e-01,118828,1.000000,1.733972e-04
ffd658295a9d2112c86c67c86118448c1e40f91eaeba5b5cbba7caf331e98322,0,NaN,20,9.999942e-01,118828,1.000000,1.155982e-04


In [20]:
# Calculate sum of 'gini' column values.
g_not_normalized = df['gini'].sum()
g_not_normalized

2700204.3319594306

Let's put everything togheter in two functions to calculate **d** and **g_not_normalized**:

In [21]:
def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    
def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

In [22]:
print('D = {}'.format(top_four_percent_captured(y_true, y_pred)))
print('g_not_normalized = {}'.format(weighted_gini(y_true, y_pred)))

D = 0.35181943649644865
g_not_normalized = 2700204.3319594306


##### Calculating G (Normalized Weighted Gini):

Let's break down the steps to caluclate Normalized Weighted Gini:
- Create y_true_pred dataframe by modifying 'target' column name to 'prediction' column name in y_true dataframe.
- Return weighted_gini(y_true, y_pred) divided by weighted_gini(y_true, y_true_pred)

In [23]:
y_true_pred = y_true.rename(columns={'target': 'prediction'})
y_true_pred

,prediction
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0
...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0


In [24]:
weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

0.7939814297196168

Let's put everything togheter in one functions to calculate **g_normalized**:

In [25]:
def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    y_true_pred = y_true.rename(columns={'target': 'prediction'})
    return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

Now that we have calculated **D** and **G**, we can measure the evaluation metric, **M**.

In [26]:
g = normalized_weighted_gini(y_true, y_pred)
d = top_four_percent_captured(y_true, y_pred)

0.5 * (g + d)

0.5729004331080327

Now let's put everything in one place:

In [27]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [28]:
amex_metric(y_true, y_pred)

0.5729004331080327